### Download Flowers dataset

In [ ]:
#You can download the data manually as well instead of using 'wget'
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/17/17flowers.tgz --quiet

In [ ]:
#Check if file is downloaded
!ls -l

In [ ]:
#Unzip the data
!tar -xf 17flowers.tgz

In [ ]:
#Check how data is organized
!ls -l 17flowers

### Build batch generator

In [ ]:
import tensorflow as tf

In [ ]:
#Define some parameters
img_size = 60
img_depth = 3

Create an ImageDataGenerator object, it can also split data between train and test.

In [ ]:
#ImageDataGenerator declaration with 20% data as test (80% for training)
img_generator= tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

ImageDataGenerator can read images directory and also resize them if needed

In [ ]:
#Build training generator. 
train_generator = img_generator.flow_from_directory('flower_photos',
                                                    target_size=(img_size, img_size),
                                                    subset='training',
                                                    batch_size=64)

In [ ]:
#Build test generator
test_generator = img_generator.flow_from_directory('flower_photos',
                                                   target_size=(img_size, img_size),                                                   
                                                   subset='validation',
                                                   batch_size=64)

In [ ]:
type(train_generator)

ImageDataGenerator returns 64 images and their labels

In [ ]:
#Lets check the features (images) and Labels (flower class) returned by ImageDataGenerator
X, y = next(train_generator)

print('Input features shape', X.shape)
print('Actual labels shape', y.shape)

In [ ]:
y[0]

In [ ]:
import numpy as np
np.unique(X[0])

In [ ]:
train_generator.class_indices

In [ ]:
X[0]

### Build CNN Model

In [ ]:
#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()

#normalize data
model.add(tf.keras.layers.BatchNormalization(input_shape=(img_size,img_size,3,)))

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Add Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add Dropout
model.add(tf.keras.layers.Dropout(0.25))

#Add Output Layer
model.add(tf.keras.layers.Dense(5, activation='softmax'))

In [ ]:
#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Model Summary
model.summary()

### Train the model

In [ ]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('flowers.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [ ]:
2939//64

In [ ]:
model.fit(train_generator,
          epochs=200,
          steps_per_epoch= 2939//64,  #Number of batches per epoch
          validation_data=test_generator,
          validation_steps = 731//64, 
          callbacks=[model_checkpoint]) #Number of test images//batch_size